<a href="https://colab.research.google.com/github/sakibchowdhury131/sentimentAnalysisDL_v2.0/blob/main/sentimentAnalysis_test1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
! git clone https://github.com/gitdevqiang/Covid-19-Sentiment-Analysis.git

Cloning into 'Covid-19-Sentiment-Analysis'...
remote: Enumerating objects: 87, done.
remote: Counting objects: 100% (87/87), done.
remote: Compressing objects: 100% (83/83), done.
remote: Total 87 (delta 26), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (87/87), done.


In [18]:
import os
os.chdir('/content/Covid-19-Sentiment-Analysis/Dataset')
os.listdir()

['Covid-19-Sentiment-Analysis',
 'train.csv',
 'test_content.csv',
 'val_content.csv']

In [19]:
import pandas as pd
def load_data(csv_file, columnNames = ['Tweet', 'Labels']): 
    dataset = pd.read_csv(csv_file)
    return dataset[columnNames]

In [26]:
df = load_data('train.csv')

In [93]:
df[0:10]

,Tweet,Labels
0,NO JOKE I WILL HOP ON A PLANE RN! (Well after ...,0 10
1,BanMediaHouse whose is responsible for spreadi...,6
2,Im waiting for someone to say to me that all t...,3 4
3,He is a liar. Proven day night. Time again. Li...,6
4,"NEW: U.S. CoronaVirus death toll reaches 4,000...",8
5,Coronavirus impact Govt extends I-T deadlines ...,5 8
6,"42,000 people might have died in China from Co...",6 7 8
7,Dear Chinese! Kindly cook your bat thoroughly ...,5 10
8,This is how the govt of kenya is checking the ...,3 6 9
9,My mental health hasn't suffered at all under ...,10


In [28]:
! pip install num2words
import nltk
nltk.download('punkt')
import re
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [33]:
corpus = []

for i in range (len(df)):
  review = re.sub('[^a-zA-Z@]', ' ', df['Tweet'][i])

  review = review.lower()
  review = review.split()
  bog = []
  for j in range (len(review)):
    if review[j].startswith('@') == False:
      bog.append(review[j])
  bog = ' '.join(bog) 
  corpus.append(bog)

In [94]:
corpus[0:10]

['no joke i will hop on a plane rn well after covid lol',
 'banmediahouse whose is responsible for spreading fake and communal stories in this pandemic corona situation',
 'im waiting for someone to say to me that all this corona thing is just an april fools joke',
 'he is a liar proven day night time again lies when the truth will do covid',
 'new u s coronavirus death toll reaches after nearly new deaths were reported today bno news covid coronavirusoutbreak',
 'coronavirus impact govt extends i t deadlines related to sections c d',
 'people might have died in china from covid but china was underreporting according to sources',
 'dear chinese kindly cook your bat thoroughly next time regards covid coronavirusupdates coronavirus',
 'this is how the govt of kenya is checking the temperatures of covid and saying it is still low mtashangaa sana',
 'my mental health hasn t suffered at all under the coronavirus quarantine ha ha april fools']

In [43]:
print(len(df['Labels']))

5000


In [42]:
import numpy as np
label = df['Labels'][0]
label = label.split()
print(label)

['0', '10']


In [76]:
def processLabels(labelsDF):
  num_class = 11
  dfLen = len(labelsDF)
  Y = np.zeros((dfLen, num_class), dtype = 'int')

  for i in range(len(labelsDF)):
    label = labelsDF[i]
    label = label.split()
    for word in label:
      x = int(word)
      Y[i, x] = 1
  return Y


In [77]:
Y = processLabels(labelsDF = df['Labels'])

In [82]:
texts = corpus

In [84]:
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences
from nltk.tokenize import word_tokenize


tokens = []
for line in texts:
    words = word_tokenize(line)
    tokens.append(words)

tokenizer_obj = Tokenizer()
tokenizer_obj.fit_on_texts(tokens)
sequences = tokenizer_obj.texts_to_sequences(tokens)

In [95]:
tokens[0:10]

[['no',
  'joke',
  'i',
  'will',
  'hop',
  'on',
  'a',
  'plane',
  'rn',
  'well',
  'after',
  'covid',
  'lol'],
 ['banmediahouse',
  'whose',
  'is',
  'responsible',
  'for',
  'spreading',
  'fake',
  'and',
  'communal',
  'stories',
  'in',
  'this',
  'pandemic',
  'corona',
  'situation'],
 ['im',
  'waiting',
  'for',
  'someone',
  'to',
  'say',
  'to',
  'me',
  'that',
  'all',
  'this',
  'corona',
  'thing',
  'is',
  'just',
  'an',
  'april',
  'fools',
  'joke'],
 ['he',
  'is',
  'a',
  'liar',
  'proven',
  'day',
  'night',
  'time',
  'again',
  'lies',
  'when',
  'the',
  'truth',
  'will',
  'do',
  'covid'],
 ['new',
  'u',
  's',
  'coronavirus',
  'death',
  'toll',
  'reaches',
  'after',
  'nearly',
  'new',
  'deaths',
  'were',
  'reported',
  'today',
  'bno',
  'news',
  'covid',
  'coronavirusoutbreak'],
 ['coronavirus',
  'impact',
  'govt',
  'extends',
  'i',
  't',
  'deadlines',
  'related',
  'to',
  'sections',
  'c',
  'd'],
 ['people',


In [86]:
def getMaxLen(tokens):
  maxLen = 0
  for token in tokens:
    if len(token)>maxLen:
      maxLen = len(token)

  return maxLen
  

In [90]:
tokenizer_word_index = tokenizer_obj.word_index
max_length = getMaxLen(tokens)
print(max_length)
review_pad = pad_sequences(sequences, maxlen = max_length)

# train test split

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(review_pad, Y, test_size = 0.20, random_state = 42)


31
